## Random Forest Approach



### Setup

In [1]:
import pandas as pd
import imp
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

import os 
from pathlib import Path


/tmp/ipykernel_7727/4288243911.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
# read dynamic path
base_dir = Path(os.getcwd())/"implementation"
data_dir = base_dir / "data/source/"
result_dir = base_dir / "data/results/"

### load & prepare dataset

the following code needs to be adapted for each protein-ligand complex individually

In [3]:
rf_data_raw = pd.read_csv(data_dir / "ACHE/ache_pca.csv")

rf_data_raw

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,43,44,45,46,47,48,49,50,51,LABEL
0,0,2.775103,1.569092,-1.633093,0.509205,-0.365362,0.464463,-1.344698,0.340212,1.195709,...,0.028038,0.472668,0.000417,0.032610,0.321647,0.373303,0.378937,0.053533,0.146973,active
1,1,1.160143,0.745342,3.626592,-0.482753,-0.501234,-0.702610,0.148980,1.191717,-0.315544,...,0.152880,-0.211698,-0.036652,0.121264,0.091523,0.055746,-0.028294,0.070177,0.144679,active
2,2,1.347487,-1.066134,0.930806,-0.326445,1.909936,0.292744,0.015491,0.427259,0.980201,...,0.217302,-0.109417,0.019410,-0.095346,-0.155197,0.019081,-0.141183,-0.306460,-0.062238,active
3,3,4.217589,-0.574677,1.399565,1.361755,-1.065899,0.487160,0.041603,0.652388,0.670705,...,0.157739,-0.189402,0.158445,0.220310,0.021845,-0.099789,0.070753,0.123486,0.066222,active
4,4,2.269227,-0.097871,0.590080,0.325115,1.649822,0.056987,-0.778049,0.349009,0.760214,...,0.067016,-0.050732,-0.009238,0.209047,-0.014614,-0.088393,0.046919,-0.085897,0.123251,active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,998,-0.609818,-0.656158,-0.131409,0.554176,-0.437736,-0.673396,-0.254374,-0.307080,-0.299093,...,-0.102843,0.016278,0.070193,-0.382453,0.163873,-0.657440,-0.134046,0.768515,0.256114,active
999,999,-0.418320,-0.805650,-0.109816,-0.376300,-0.549992,-0.458199,0.583562,0.367584,0.043453,...,-0.086251,-0.012426,0.085909,0.038385,0.079218,0.017346,0.007403,-0.071999,-0.063954,active
1000,1000,-1.637342,-0.955758,0.168724,0.684614,-0.258857,-0.898800,0.085315,0.578703,0.398927,...,-0.576817,0.119175,-0.115178,0.025776,-0.052349,0.095234,0.116501,0.038220,0.091057,active
1001,1001,0.573375,-0.944395,-0.090789,-0.004957,-0.256222,-0.095932,0.025505,-0.122203,0.328594,...,-0.200537,0.019137,-0.007516,0.055282,-0.002256,0.076485,-0.001794,-0.038076,-0.072265,inactive


In [4]:
lookup = {'inactive':0,'active':1}

rf_data = {'data': np.array(rf_data_raw.iloc[:, 2:-1]),
             'target': np.array([lookup[y] for y in rf_data_raw.iloc[0:,-1]]),
             'feature_names': rf_data_raw.columns[2:-1],
             'target_names': ['inactive', 'active']}


split into train- and test-set

In [5]:
X_train, X_test, y_train, y_test = train_test_split(rf_data['data'], rf_data['target'],
                                                    test_size=0.3, random_state=4232)

### Apply Random Forrest

In [6]:
parameters = {'max_depth': [i for i in range(3, 28,3)]}
rf_models = GridSearchCV(RandomForestClassifier(n_estimators=1001, oob_score=True), parameters, cv=20, n_jobs=-1)


Fit model with the provided parameters

In [7]:
rf_models.fit(X_train, y_train)

GridSearchCV(cv=20,
             estimator=RandomForestClassifier(n_estimators=1001,
                                              oob_score=True),
             n_jobs=-1,
             param_grid={'max_depth': [3, 6, 9, 12, 15, 18, 21, 24, 27]})

In [8]:
pd.DataFrame(rf_models.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split13_test_score,split14_test_score,split15_test_score,split16_test_score,split17_test_score,split18_test_score,split19_test_score,mean_test_score,std_test_score,rank_test_score
0,9.315712,0.670884,0.160378,0.035092,3,{'max_depth': 3},0.777778,0.750000,0.828571,0.685714,...,0.714286,0.771429,0.828571,0.742857,0.657143,0.800000,0.628571,0.723532,0.063054,9
1,11.237012,0.651126,0.126995,0.028401,6,{'max_depth': 6},0.833333,0.750000,0.857143,0.742857,...,0.771429,0.742857,0.857143,0.714286,0.742857,0.771429,0.742857,0.786310,0.053004,8
2,15.151802,0.703255,0.128190,0.038726,9,{'max_depth': 9},0.861111,0.750000,0.857143,0.714286,...,0.800000,0.742857,0.828571,0.742857,0.742857,0.828571,0.742857,0.797698,0.059077,5
3,12.170903,0.555166,0.125162,0.027740,12,{'max_depth': 12},0.861111,0.722222,0.857143,0.800000,...,0.800000,0.742857,0.885714,0.742857,0.742857,0.800000,0.742857,0.800595,0.056776,4
4,10.867641,0.396476,0.111711,0.026993,15,{'max_depth': 15},0.888889,0.750000,0.828571,0.771429,...,0.771429,0.714286,0.857143,0.714286,0.714286,0.800000,0.714286,0.797659,0.063119,7
5,10.960136,0.386406,0.127492,0.023878,18,{'max_depth': 18},0.833333,0.777778,0.857143,0.742857,...,0.771429,0.714286,0.857143,0.714286,0.742857,0.771429,0.742857,0.797698,0.056365,5
6,11.046058,0.400207,0.132296,0.034390,21,{'max_depth': 21},0.833333,0.777778,0.857143,0.771429,...,0.771429,0.742857,0.857143,0.714286,0.742857,0.800000,0.742857,0.801984,0.053019,3
7,14.482358,1.130025,0.165251,0.043491,24,{'max_depth': 24},0.833333,0.750000,0.828571,0.771429,...,0.800000,0.714286,0.857143,0.742857,0.714286,0.828571,0.771429,0.803452,0.052346,2
8,24.011092,2.531673,0.341947,0.087923,27,{'max_depth': 27},0.833333,0.777778,0.800000,0.771429,...,0.800000,0.742857,0.857143,0.742857,0.714286,0.828571,0.771429,0.807698,0.050916,1


In [9]:
rf_models.best_score_

0.8076984126984126

In [10]:
rf_models.best_params_

{'max_depth': 27}

### Score

In [11]:
prediction = rf_models.best_estimator_.predict(X_test)
rf_models.best_estimator_.score(X_test,y_test)

0.7906976744186046

### Prepare Data for evaluation

In [12]:
result_df = pd.DataFrame(columns=rf_data_raw.columns[:-1])

for i, row in enumerate(X_test):
    data = [i,i]
    data.extend(row)
    result_df.loc[i+1] = data

result_df["LABEL"] = y_test
result_df["PRED"] = prediction

result_df.to_csv(result_dir/"ACHE/fe_pca_rf.csv",encoding="utf-8")